In [1]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.catboost_multiclass import CatBoostMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="Catb")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def catboost_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model = CatBoostMulticlass(
            **cfg_models["catboost"],
    ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=catboost_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "logreg_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)

2026-01-08 15:48:11.742 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:36 - Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-08 15:48:11.744 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:37 - Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts
2026/01/08 15:48:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/08 15:48:12 INFO mlflow.store.db.utils: Updating database tables
2026/01/08 15:48:12 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/08 15:48:12 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/08 15:48:12 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/08 15:48:12 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026-01-08 15:48:12.356 | INFO     | fertilizer_recommender.infrastructure.tracking.mlflow_setup:configur

0:	learn: 1.9453194	total: 401ms	remaining: 4m
100:	learn: 1.9288982	total: 31.5s	remaining: 2m 35s
200:	learn: 1.9195964	total: 1m 4s	remaining: 2m 8s
300:	learn: 1.9112166	total: 1m 49s	remaining: 1m 48s
400:	learn: 1.9032716	total: 2m 20s	remaining: 1m 9s
500:	learn: 1.8955490	total: 2m 52s	remaining: 34.1s


2026-01-08 15:51:41.190 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:95 - [Fold 1] Prédiction et calcul du top-3


599:	learn: 1.8884225	total: 3m 22s	remaining: 0us


2026-01-08 15:51:43.344 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 1] Score MAP@3 = 0.3191
2026-01-08 15:51:43.345 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold': 0.3191133333333964}
2026-01-08 15:51:43.345 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold': 0.3191133333333964}
2026-01-08 15:51:43.477 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:86 - [Fold 2] Démarrage (train=600000 obs, val=150000 obs)
2026-01-08 15:51:43.478 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:91 - [Fold 2] Entraînement du modèle


...

599:	learn: 1.8887100	total: 3m 30s	remaining: 0us


2026-01-08 16:03:35.643 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 4] Score MAP@3 = 0.3200
2026-01-08 16:03:35.644 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold': 0.32000666666673416}
2026-01-08 16:03:35.644 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold': 0.32000666666673416}
2026-01-08 16:03:35.919 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:86 - [Fold 5] Démarrage (train=600000 obs, val=150000 obs)
2026-01-08 16:03:35.920 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:91 - [Fold 5] Entraînement du modèle


0:	learn: 1.9453478	total: 618ms	remaining: 6m 10s
100:	learn: 1.9289165	total: 43.8s	remaining: 3m 36s
200:	learn: 1.9198391	total: 1m 32s	remaining: 3m 4s
300:	learn: 1.9114347	total: 2m 30s	remaining: 2m 29s
400:	learn: 1.9035912	total: 3m 9s	remaining: 1m 33s
500:	learn: 1.8959983	total: 3m 45s	remaining: 44.6s
599:	learn: 1.8889384	total: 4m 39s	remaining: 0us


2026-01-08 16:08:22.780 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:95 - [Fold 5] Prédiction et calcul du top-3
2026-01-08 16:08:25.818 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 5] Score MAP@3 = 0.3201
2026-01-08 16:08:25.819 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold': 0.3201022222222902}
2026-01-08 16:08:25.820 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold': 0.3201022222222902}
2026-01-08 16:08:25.833 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_mean': 0.31977355555562237}
2026-01-08 16:08:25.834 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_mean': 0.319773555

Fold MAP@3: [0.3191133333333964, 0.32001333333340265, 0.3196322222222885, 0.32000666666673416, 0.3201022222222902]
Mean MAP@3: 0.31977355555562237


In [ ]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.lightgbm_multiclass import LightGBMMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="Lightgbm")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def lightgbm_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model=LightGBMMulticlass(
            num_class=len(y.unique()),
            **cfg_models["lightgbm"],
        ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=lightgbm_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "logreg_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)

In [ ]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.xgboost_multiclass import XGBoostMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="Lightgbm")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def lightgbm_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model=XGBoostMulticlass(
            num_class=len(y.unique()),
            **cfg_models["xgboost"],
        ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=lightgbm_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "logreg_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)